In [32]:
key_words = ['INDI', 'NAME', 'SEX', 'BIRT', 'DEAT', 'FAMC', 'FAMS', 'FAM', 'MARR', 'HUSB', 'WIFE', 'CHIL', 'DIV', 'DATE', 'HEAD', 'TRLR', 'NOTE']

In [33]:
file = open('family.ged', 'r')

In [34]:
for line in file:
    print("--> " + line.strip())
    line_words = line.split()
    # get the first element since that is the tag of line
    level = line_words[0].strip()
    try:
        if line_words[2].strip() == "INDI" or line_words[2].strip() == "FAM":
            line_tag = line_words[2].strip()
            arguments = line_words[1].strip()
        else:
            line_tag = line_words[1].strip()
            arguments = line.split(line_tag)[-1].strip() 
    except:
        line_tag = line_words[1].strip()
        arguments = line.split(line_tag)[-1].strip()
    
    # check if that is present in the keywords
    if line_tag in key_words:
        valid = 'Y'
    else:
        valid = 'N'
    
    
    if len(arguments) > 0:
        print("<-- " + level + "|" + line_tag + "|" + valid + "|" + arguments)
    else:
        print("<-- " + level + "|" + line_tag + "|" + valid)


--> 0 NOTE @N1@
<-- 0|NOTE|Y|@N1@
--> 1 CONC Yashvardhan Sharma
<-- 1|CONC|N|Yashvardhan Sharma
--> 0 HEAD
<-- 0|HEAD|Y
--> 1 SOUR Family Echo
<-- 1|SOUR|N|Family Echo
--> 2 WWW http://www.familyecho.com/
<-- 2|WWW|N|http://www.familyecho.com/
--> 1 FILE My Family
<-- 1|FILE|N|My Family
--> 1 DATE 16 SEP 2023
<-- 1|DATE|Y|16 SEP 2023
--> 1 DEST ANSTFILE
<-- 1|DEST|N|ANSTFILE
--> 1 GEDC
<-- 1|GEDC|N
--> 2 VERS 5.5.1
<-- 2|VERS|N|5.5.1
--> 2 FORM LINEAGE-LINKED
<-- 2|FORM|N|LINEAGE-LINKED
--> 1 SUBM @I1@
<-- 1|SUBM|N|@I1@
--> 2 NAME Yashvardhan Sharma
<-- 2|NAME|Y|Yashvardhan Sharma
--> 1 SUBN
<-- 1|SUBN|N
--> 1 CHAR UTF-8
<-- 1|CHAR|N|UTF-8
--> 0 @I1@ INDI
<-- 0|INDI|Y|@I1@
--> 1 NAME Yashvardhan /Sharma/
<-- 1|NAME|Y|Yashvardhan /Sharma/
--> 2 GIVN Yashvardhan
<-- 2|GIVN|N|Yashvardhan
--> 2 SURN Sharma
<-- 2|SURN|N|Sharma
--> 2 _MARNM Sharma
<-- 2|_MARNM|N|Sharma
--> 1 SEX M
<-- 1|SEX|Y|M
--> 1 BIRT
<-- 1|BIRT|Y
--> 2 DATE 7 MAR 2002
<-- 2|DATE|Y|7 MAR 2002
--> 1 FAMC @F1@
<-- 1|FAMC|Y